In [5]:
#1.import libraries
import pandas as pd
import numpy as np
from pprint import pprint
import random


mnod = []
leaves = []

dataset = pd.read_csv('adult.train.10k.discrete',
                      names=['class','workclass','education','martial-status','occupation','relationship','race','sex','native-country'])#Import all columns omitting the fist which consists the names of the animals





dataset = dataset[['workclass','education','martial-status','occupation','relationship','race','sex','native-country','class']]



testing_data = pd.read_csv('adult.test.10k.discrete',
                      names=['class','workclass','education','martial-status','occupation','relationship','race','sex','native-country'])#Import all columns omitting the fist which consists the names of the animals





testing_data = testing_data[['workclass','education','martial-status','occupation','relationship','race','sex','native-country','class']]


dataset_30 = dataset.sample(frac = 0.3)
dataset_40 = dataset.sample(frac = 0.4)
dataset_50 = dataset.sample(frac = 0.5)
dataset_60 = dataset.sample(frac = 0.6)
dataset_70 = dataset.sample(frac = 0.7)

#2.Entropy
def entropy(col):
    
    elements,counts = np.unique(col,return_counts = True)
    entropy = np.sum([(-counts[i]/np.sum(counts))*np.log2(counts[i]/np.sum(counts)) for i in range(len(elements))])
    return entropy




#3.calculate IG 
def InfoGain(data,split_attribute_name,target_name="class"):
   
   
    total_entropy = entropy(data[target_name])
    

    
   
    vals,counts= np.unique(data[split_attribute_name],return_counts=True)
    

    Weighted_Entropy = np.sum([(counts[i]/np.sum(counts))*entropy(data.where(data[split_attribute_name]==vals[i]).dropna()[target_name]) for i in range(len(vals))])
    print(Weighted_Entropy)

    Information_Gain = total_entropy - Weighted_Entropy
    return Information_Gain
       


#ID3 algorithm
def ID3(data,originaldata,features,target_attribute_name="class",parent_node_class = None):
    
    
    if len(np.unique(data[target_attribute_name])) <= 1:
        return np.unique(data[target_attribute_name])[0]
  
    elif len(features) ==0:
        return parent_node_class
    
    
    
    else:
        
        parent_node_class = np.unique(data[target_attribute_name])[np.argmax(np.unique(data[target_attribute_name],return_counts=True)[1])]
        
        
        item_values = [InfoGain(data,feature,target_attribute_name) for feature in features] 
        best_feature_index = np.argmax(item_values)
        best_feature = features[best_feature_index]
        
       
        tree = {best_feature:{}}
        
        
        
        features = [i for i in features if i != best_feature]
        
      
        
        for value in np.unique(data[best_feature]):
            value = value
            
            sub_data = data.where(data[best_feature] == value).dropna()
            
            
            
            subtree = ID3(sub_data,dataset,features,target_attribute_name,parent_node_class)
            
           
            tree[best_feature][value] = subtree
            
        return(tree)    
                



    
    
def predict(ex,tree,default = 1):
   
    
    
    for key in list(ex.keys()):
        if key in list(tree.keys()):
            
            try:
                result = tree[key][ex[key]] 
            except:
                return default
  
            
            result = tree[key][ex[key]]
            
            if isinstance(result,dict):
                return predict(ex,result)

            else:
                return result

        
        


def test(data,tree):
    
    ex_test = data.iloc[:,:-1].to_dict(orient = "records")
    
   
    predicted = pd.DataFrame(columns=["predicted"]) 
    
 
    for i in range(len(data)):
        predicted.loc[i,"predicted"] = predict(ex_test[i],tree,1.0) 
    print('The prediction accuracy is: ',(np.sum(predicted["predicted"] == data["class"])/len(data))*100,'%')



    
def count_items(d):
   
    global mnod
    global leaves
    for k, v in d.items():
        if k not in mnod:
            
            mnod.append(k)
        if isinstance(v, dict):
            count_items(v)
        else:
            leaves.append(v) 
            
# i learned it from youtube that was so hard for the first time as  an electrical student             
# def filter_df(df, question):
#     feature, _, value = question.split()
#     df_yes = df[df[feature] <= float(value)]
#     df_no  = df[df[feature] >  float(value)]
    
#     return df_yes, df_no


# def pruning_result(tree, df_train, df_val):

#     leaf = df_train.label.value_counts().index[0]
#     errors_leaf = sum(df_val.label != leaf)
#     errors_decision_node = sum(df_val.label != make_predictions(df_val, tree))

#     if errors_leaf <= errors_decision_node:
#         return leaf
#     else:
#         return tree
    
# def post_pruning(tree, df_train, df_val):
    
#     question = list(tree.keys())[0]
#     yes_answer, no_answer = tree[question]

#     
#     if not isinstance(yes_answer, dict) and not isinstance(no_answer, dict):
#         return pruning_result(tree, df_train, df_val)
        
#     
#     else:
#         df_train_yes, df_train_no = filter_df(df_train, question)
#         df_val_yes, df_val_no = filter_df(df_val, question)
        
#         if isinstance(yes_answer, dict):
#             yes_answer = post_pruning(yes_answer, df_train_yes, df_val_yes)
            
#         if isinstance(no_answer, dict):
#             no_answer = post_pruning(no_answer, df_train_no, df_val_no)
            
#         tree = {question: [yes_answer, no_answer]}
    
#         return pruning_result(tree, df_train, df_val)
           
            




# #time to let the machine learn :D!

# tree_30 = ID3(dataset_30,dataset_30,dataset_30.columns[:-1])
# print("predict for 30% of data:")
# test(testing_data,tree_30)
# count_items(tree_30)
# print("number of nods for 30% of data:")
# print(round((len(mnod)+len(leaves)+1)/2))
# mnod = []
# leaves = []

# tree_40 = ID3(dataset_40,dataset_40,dataset_40.columns[:-1])
# print("predict for 40% of data:")
# test(testing_data,tree_40)
# count_items(tree_40)
# print("number of nods for 40% of data:")
# print(round((len(mnod)+len(leaves)+1)/2))
# mnod = []
# leaves = []

# tree_50 = ID3(dataset_50,dataset_50,dataset_50.columns[:-1])
# print("predict for 50% of data:")
# test(testing_data,tree_50)
# count_items(tree_50)
# print("number of nods for 50% of data:")
# print(round((len(mnod)+len(leaves)+1)/2))
# mnod = []
# leaves = []

# tree_60 = ID3(dataset_60,dataset_60,dataset_60.columns[:-1])
# print("predict for 60% of data:")
# test(testing_data,tree_60)
# count_items(tree_60)
# print("number of nods for 60% of data:")
# print(round((len(mnod)+len(leaves)+1)/2))
# mnod = []
# leaves = []

# tree_70 = ID3(dataset_70,dataset_70,dataset_70.columns[:-1])
# print("predict for 70% of data:")
# test(testing_data,tree_70)
# count_items(tree_70)
# print("number of nods for 70% of data:")
# print(round((len(mnod)+len(leaves)+1)/2))
# mnod = []
# leaves = []

# tree = ID3(dataset,dataset,dataset.columns[:-1])
# print("predict for 100% of data:")
# test(testing_data,tree)
# count_items(tree)
# print("number of nods for 100% of data:")
# print(round((len(mnod)+len(leaves)+1)/2))
InfoGain(dataset,sex,target_name="class")





NameError: name 'sex' is not defined